In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/processed/processed.csv')

In [3]:
df.shape

(4132, 7)

In [4]:
df.sample(3)

,brand,model_year,kms_driven,owner,location,price,model_age
2773,KTM,2016,7488,1st,Delhi,120000,5
2889,Hero,2018,6750,1st,other,22500,3
175,Yamaha,2015,36000,1st,other,37000,6


In [5]:
df = df.drop(['location'],axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4132 entries, 0 to 4131
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   brand       4132 non-null   object
 1   model_year  4132 non-null   int64 
 2   kms_driven  4132 non-null   int64 
 3   owner       4132 non-null   object
 4   price       4132 non-null   int64 
 5   model_age   4132 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 193.8+ KB


## Category Encoding

In [7]:
owner_categories = list(df.owner.unique()[::-1])
owner_categories

['5th', '4th', '3rd', '2nd', '1st']

In [8]:
df.sample()

,brand,model_year,kms_driven,owner,price,model_age
1971,Bajaj,2020,11000,1st,80000,1


In [9]:
# ## Change the owner category to numerical encoding
# owners_ranks = {
#     '1st': 5,
#     '2nd': 4,
#     '3rd': 3,
#     '4th': 2,
#     '5th': 1,
    
# }

# df['owner'] = df['owner'].apply(lambda x: owners_ranks.get(x,1))

# df.info()

In [10]:
X = df.drop('price',axis=1)
y = df['price']

In [11]:
y = y.apply(np.log1p)

In [12]:
X.sample(3)

,brand,model_year,kms_driven,owner,model_age
516,Royal Enfield,2015,12100,1st,6
3743,Bajaj,2009,43000,2nd,12
3079,Bajaj,2013,20000,1st,8


In [13]:
# ['5th','4th','3rd','2nd','1st']

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,MinMaxScaler


from sklearn.base import BaseEstimator, TransformerMixin
# Custom transformer to normalize model_year
class ModelYearTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        super().__init__()

    def fit(self,X,y = None):
        return self

    def transform(self,X,y=None):
        X_ = X.copy()
        X_.model_year = X_.model_year - 2010

        return X_



category_transformer = ColumnTransformer([
    ("brand_ohe",OneHotEncoder(dtype=np.int16,sparse=False,handle_unknown='ignore'),[0]),
    # ("model_year_normalizer",ModelYearTransformer(),[1]),
    ("kms_driven_min_max_scaler",MinMaxScaler(),[2]),
    ("owner_ordinal_enc",OrdinalEncoder(categories='auto',handle_unknown='ignore',dtype=np.int16),[3]),
    # ("location_ohe",OneHotEncoder(dtype=np.int16, sparse=False,handle_unknown='ignore'),[4]),
],remainder='passthrough')



In [15]:
# Scale the price data
# y = MinMaxScaler().fit_transform([y])[0]

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=1234)

## Model Building

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn import set_config

set_config(display='diagram')

In [18]:

def build_pipeline_with_estimator(estimator):
    return Pipeline([
    ('category_transformer',category_transformer),
    ('estimator',estimator),
])


In [19]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import cross_val_score
# MAPE
def mape(targets, predictions):
    return np.mean(np.abs((targets - predictions)) / targets) * 100

# Adjusted R^2
def adj_r2(ind_vars, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = ind_vars.shape[0]
    k = ind_vars.shape[1]
    return 1-((1-r2)*(n-1)/(n-k-1))

# Model performance check
def model_perf(model, inp, out):

    y_pred = model.predict(inp)
    y_act = out.values

    cross_val = cross_val_score(model, inp,out,cv=10)
    

    return pd.DataFrame({
                "RMSE": np.sqrt(mean_squared_error(y_act, y_pred)),
                "MAE": mean_absolute_error(y_act, y_pred),
                "MAPE": mape(y_act, y_pred),
                "R^2": r2_score(y_act, y_pred),
                "Adjusted R^2": adj_r2(inp, y_act, y_pred),
                "Cross Val Score (Mean)": cross_val.mean()
           }, index=[0])

In [20]:
# linear_regressor = build_pipeline_with_estimator(LinearRegression())
# scores = []
# for i in range(1000):
#     X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=1)

#     linear_regressor.fit(X_train,y_train)

#     y_pred = linear_regressor.predict(X_test)

#     s = r2_score(y_test,y_pred)

#     scores.append(s)

# print(scores)
    
# liner_regressor.fit(X_train,y_train)

# print('Linear Regression Train Performance.\n')
# print(model_perf(liner_regressor,X_train,y_train))

# print('Linear Regression Test Performance.\n')
# print(model_perf(liner_regressor,X_test,y_test))

## LinearRegression

In [21]:
liner_regressor = build_pipeline_with_estimator(LinearRegression())

params = {}

liner_regressor = GridSearchCV(liner_regressor,params,cv=10)

liner_regressor.fit(X_train,y_train)

print('Linear Regression Train Performance.\n')
print(model_perf(liner_regressor,X_train,y_train))

print('Linear Regression Test Performance.\n')
print(model_perf(liner_regressor,X_test,y_test))

Linear Regression Train Performance.

       RMSE       MAE      MAPE      R^2  Adjusted R^2  Cross Val Score (Mean)
0  0.477585  0.362975  3.361202  0.54204      0.541346                0.537353
Linear Regression Test Performance.

       RMSE       MAE      MAPE       R^2  Adjusted R^2  \
0  0.477753  0.366346  3.375539  0.540805      0.538008   

   Cross Val Score (Mean)  
0               -0.241627  


## RandomForest

In [22]:
from sklearn.ensemble import RandomForestRegressor

model = build_pipeline_with_estimator(RandomForestRegressor())

# params = {
#     'estimator__criterion': ['gini','entropy'],
#     'estimator__n_estimators': [100],
#     'estimator__max_depth': [5,10,15,20,25],
#     'estimator__min_samples_split': range(2,11),
#     'estimator__max_features': ['auto','sqrt','log2'],
# }

# model = GridSearchCV(model,params,cv=10)

model.fit(X_train,y_train)

print('RandomForest Train Performance.\n')
print(model_perf(model,X_train,y_train))

print('RandomForest Test Performance.\n')
print(model_perf(model,X_test,y_test))

RandomForest Train Performance.

       RMSE       MAE      MAPE       R^2  Adjusted R^2  \
0  0.230062  0.152402  1.399345  0.893729      0.893567   

   Cross Val Score (Mean)  
0                  0.5461  
RandomForest Test Performance.

       RMSE       MAE      MAPE      R^2  Adjusted R^2  Cross Val Score (Mean)
0  0.459848  0.324745  2.967276  0.57458      0.571989                 0.47915


## XGBoost

In [23]:
from xgboost import XGBRegressor

xgboost = build_pipeline_with_estimator(XGBRegressor())

xgboost.fit(X_train,y_train)

print('xgboost Train Performance.\n')
print(model_perf(xgboost,X_train,y_train))

print('xgboost Test Performance.\n')
print(model_perf(xgboost,X_test,y_test))

xgboost Train Performance.

       RMSE       MAE      MAPE       R^2  Adjusted R^2  \
0  0.279987  0.199967  1.840092  0.842602      0.842363   

   Cross Val Score (Mean)  
0                0.578851  
xgboost Test Performance.

       RMSE       MAE      MAPE       R^2  Adjusted R^2  \
0  0.445467  0.316339  2.889716  0.600771       0.59834   

   Cross Val Score (Mean)  
0                  0.4764  


In [24]:
from sklearn.neighbors import KNeighborsRegressor

model = build_pipeline_with_estimator(KNeighborsRegressor())

# params = {
#     'estimator__n_neighbors': [3,5,7,9],
#     'estimator__algorithm': ['ball_tree','kd_tree','brute'],
#     'estimator__leaf_size': [5,10,30,32,35]
# }

# model = GridSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('KNeighborsRegressor Train Performance.\n')
print(model_perf(model,X_train,y_train))

print('KNeighborsRegressor Test Performance.\n')
print(model_perf(model,X_test,y_test))

KNeighborsRegressor Train Performance.

       RMSE       MAE      MAPE       R^2  Adjusted R^2  \
0  0.388821  0.274376  2.511198  0.696453      0.695993   

   Cross Val Score (Mean)  
0                0.540933  
KNeighborsRegressor Test Performance.

       RMSE       MAE      MAPE       R^2  Adjusted R^2  \
0  0.479743  0.341066  3.110378  0.536971      0.534151   

   Cross Val Score (Mean)  
0                0.455039  


## GradientBoostingRegressor

In [25]:
from sklearn.ensemble import GradientBoostingRegressor

model = build_pipeline_with_estimator(GradientBoostingRegressor())

# params = {
#     'estimator__loss': ['ls','lad','huber','quantile'],
#     'estimator__learning_rate': [0.1],
#     'estimator__n_estimators': [100],
#     'estimator__criterion': ['friedman_mse','mse','mae'],
# }

# model = GridSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('GradientBoostingRegressor Train Performance.\n')
print(model_perf(model,X_train,y_train))

print('GradientBoostingRegressor Test Performance.\n')
print(model_perf(model,X_test,y_test))

GradientBoostingRegressor Train Performance.

       RMSE       MAE      MAPE       R^2  Adjusted R^2  \
0  0.419139  0.314055  2.896082  0.647271      0.646737   

   Cross Val Score (Mean)  
0                0.598646  
GradientBoostingRegressor Test Performance.

       RMSE       MAE      MAPE       R^2  Adjusted R^2  \
0  0.443787  0.329871  3.025058  0.603778      0.601365   

   Cross Val Score (Mean)  
0                0.558512  
